In [1]:
import random

NUMBER_CARDS = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
ACTION_CARDS = ["Skip", "Reverse", "Draw Two"]
WILD_CARDS = ["Wild", "Wild Draw Four"]
COLORS = ["Red", "Green", "Blue", "Yellow"]

def create_deck():
    """Creates and returns a full Uno deck with number, action, and wild cards."""
    deck = []
    # Add number cards
    for color in COLORS:
        for number in NUMBER_CARDS:
            deck.append((color, number))
            if number != "0":  
                deck.append((color, number))
    for color in COLORS:
        for action in ACTION_CARDS:
            deck.append((color, action))
            deck.append((color, action))
    for wild in WILD_CARDS:
        for _ in range(4):  
            deck.append(("Black", wild))
    return deck

def deal_cards(deck, num_players):
    """Distributes 7 cards to each player and sets up the initial discard pile."""
    random.shuffle(deck)
    player_hands = {f'Player {i+1}': [] for i in range(num_players)}
    for _ in range(7):
        for player in player_hands:
            player_hands[player].append(deck.pop())
    initial_discard = deck.pop()
    while initial_discard[1] == "Wild Draw Four":
        deck.insert(random.randint(0, len(deck)-1), initial_discard)
        random.shuffle(deck)
        initial_discard = deck.pop()
    discard_pile = [initial_discard]
    return player_hands, deck, discard_pile

def is_playable(card, top_card, current_color):
    """Check if a card can be played on top of the current top card."""
    return card[1] == top_card[1] or card[0] == current_color or card[1] in WILD_CARDS

def draw_card(player_hand, deck):
    """Player draws a card from the deck."""
    if not deck:
        print("Deck is empty. (Reshuffle not implemented.)")
        return
    player_hand.append(deck.pop())

def choose_color():
    """Randomly chooses a color."""
    return random.choice(COLORS)

def player_turn(player_hand, top_card, deck, current_color):
    print(f"Your hand: {['{}: {}'.format(i+1, card) for i, card in enumerate(player_hand)]}")
    playable_cards = [card for card in player_hand if is_playable(card, top_card, current_color)]
    
    if not playable_cards:
        print("No playable cards. Drawing a card.")
        draw_card(player_hand, deck)
        return None, current_color, None  # Include action_taken as None
    
    print("Playable cards:")
    for i, card in enumerate(playable_cards):
        print(f"{i+1}: {card}")
    
    chosen_card = None
    while chosen_card is None:
        try:
            print("Choose a card to play (enter number): ")
            choice = int(input()) - 1
            chosen_card = playable_cards[choice]
        except (ValueError, IndexError):
            print("Invalid choice. Please enter a number corresponding to a playable card.")
    
    player_hand.remove(chosen_card)
    new_color = current_color
    action_taken = None  # Initialize action_taken
    if chosen_card[1] in ["Wild", "Wild Draw Four"]:
        new_color = input_color_choice()
        action_taken = chosen_card[1]  # Action is the card type itself
    elif chosen_card[1] in ACTION_CARDS:
        action_taken = chosen_card[1]
    
    print(f"Played {chosen_card}. New color is {new_color}." if new_color != current_color else f"Played {chosen_card}.")
    return chosen_card, new_color, action_taken

def input_color_choice():
    """Asks the player to input a color choice for Wild cards."""
    print("Choose a color (Red, Green, Blue, Yellow): ")
    color_choice = input().capitalize()
    while color_choice not in COLORS:
        print("Invalid color. Choose again (Red, Green, Blue, Yellow): ")
        color_choice = input().capitalize()
    return color_choice


def check_win_condition(player_hand):
    return len(player_hand) == 0

def ai_turn(player_hand, top_card, deck, current_color):
    playable_cards = [card for card in player_hand if is_playable(card, top_card, current_color)]
    if not playable_cards:
        draw_card(player_hand, deck)
        return None, current_color, None
    chosen_card = playable_cards[0]  # Simple strategy: pick the first playable card
    player_hand.remove(chosen_card)
    new_color = current_color if chosen_card[0] != "Black" else choose_color()
    action_taken = chosen_card[1] if chosen_card[1] in ACTION_CARDS or chosen_card[1] in WILD_CARDS else None
    return chosen_card, new_color, action_taken



In [ ]:
def play_uno(num_players):
    deck = create_deck()
    player_hands, deck, discard_pile = deal_cards(deck, num_players)
    current_color = discard_pile[-1][0] if discard_pile[-1][0] != "Black" else random.choice(COLORS)
    current_player_index = 0
    play_direction = 1  # 1 for clockwise, -1 for counterclockwise
    game_over = False

    while not game_over:
        if len(player_hands[f'Player {current_player_index+1}']) == 0:
            print(f"Player {current_player_index+1} wins!")
            break

        print(f"\nCurrent top card: {discard_pile[-1]} (Color: {current_color})")
        player_hand = player_hands[f'Player {current_player_index+1}']

        if current_player_index == 0:  # Human player's turn
            print("\nYour turn:")
            played_card, current_color, action_taken = player_turn(player_hand, discard_pile[-1], deck, current_color)
        else:  # AI player's turn
            print(f"\nPlayer {current_player_index+1}'s (AI) turn:")
            played_card, current_color, action_taken = ai_turn(player_hand, discard_pile[-1], deck, current_color)
            if played_card:
                print(f"AI played {played_card}. New color: {current_color}.")

        if played_card:
            discard_pile.append(played_card)

        # Handle action_taken effects
        if action_taken == "Skip":
            current_player_index = (current_player_index + play_direction) % num_players
            print("Skipping next player.")
        elif action_taken == "Reverse":
            play_direction *= -1
            # In a 2-player game, Reverse acts like Skip, so we advance twice
            if num_players == 2:
                current_player_index = (current_player_index + 2 * play_direction) % num_players
            print("Play direction reversed.")
        elif action_taken == "Draw Two":
            next_player_index = (current_player_index + play_direction) % num_players
            for _ in range(2):
                draw_card(player_hands[f'Player {next_player_index+1}'], deck)
            print(f"Player {next_player_index+1} draws 2 cards due to Draw Two.")
            current_player_index = next_player_index  # Move to the player who just drew cards

        # Advance to the next player
        current_player_index = (current_player_index + play_direction) % num_players


play_uno(4)